In [1]:
!/d0/home/adamk/pysccnv/venv/bin/pip install pysam

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pysam
import json
import pprint
pprint = pprint.PrettyPrinter().pprint
import re
import collections
import time

# Detect polyA reads

In [3]:
def reverse_complemented_filter(line, required_length, required_content):
    if line.cigartuples[0][0] != 4: # 4 = softclip
        return False
    
    if line.cigartuples[0][1] < required_length:
        return False
    
    three_prime_sequence = line.seq[:required_length]
    if collections.Counter(three_prime_sequence)["T"] < required_content:
        return False
    
    return True

In [4]:
def standard_filter(line, required_length, required_content):
    if line.cigartuples[-1][0] != 4: # 4 = softclip
        return False
    
    if line.cigartuples[-1][1] < required_length:
        return False
    
    three_prime_sequence = line.seq[-1*required_length:]
    
    if collections.Counter(three_prime_sequence)["A"] < required_content:
        return False
    
    return True

In [5]:
def polyadenylation_reader(bam_filename, limit = -1):
    flags = collections.Counter()
    flags_after = collections.Counter()
    
    polyadenylation_reads = dict()
    
    required_soft_clip_length = 5
    required_TA_content = 4

    with pysam.AlignmentFile(bam_filename, "rb") as f:
        for i, line in enumerate(f):
            if i == limit:
                break
            if not line.cigartuples:
                continue
            if not line.cigartuples[0]:
                continue
        
            if line.flag & 0x10 == 0 and not standard_filter(line, required_soft_clip_length, required_TA_content):
                continue
            elif line.flag & 0x10 > 0 and not reverse_complemented_filter(line, required_soft_clip_length, required_TA_content):                
                continue
            
                
            tags = {i:j for i, j in line.get_tags()}
            try:
                reads_by_gene = polyadenylation_reads.setdefault(tags["GN"], {})            
                reads_by_reference_start = reads_by_gene.setdefault(line.reference_start, [])
                reads_by_reference_start.append([tags["CB"], tags["UB"], line.seq])
            except KeyError:
                pass
    
    return polyadenylation_reads

In [6]:
def serialise(o, filename):
    with open(filename, "w") as f:
        json.dump(o, f)

In [7]:
limit = 10**7
start = time.time()
polya_reads = polyadenylation_reader("5k_pbmc_protein_v3_possorted_genome_bam.bam", limit = limit)
stop = time.time()

In [8]:
runtime = stop - start

In [9]:
estimated_runtime = runtime/limit * 245409397

In [10]:
estimated_runtime

490.98455913353365

In [11]:
start = time.time()
polya_reads = polyadenylation_reader("5k_pbmc_protein_v3_possorted_genome_bam.bam")
stop = time.time()

In [12]:
stop - start

475.0309863090515

In [13]:
serialise(polya_reads, "polyadenylation_reads.json")

# Load up the polya_reads

In [14]:
def deserialise(filename):
    with open(filename, "r") as f:
        o = json.load(f)
    return o

In [15]:
polya_reads = deserialise("polyadenylation_reads.json")

In [16]:
len([key for key in polya_reads])

12127

In [17]:
counts = {}
for gene, gene_data in polya_reads.items():
    for location, sequence_data in gene_data.items():
        UMI_per_location = collections.Counter()
        for CB, UMI, seq in sequence_data:
            UMI_per_location[(CB, UMI)] += 1
        counts[gene, location] = len(UMI_per_location)

In [18]:
sorted_by_abundance = sorted([(count, gene_location) for gene_location, count in counts.items() if count > 100], reverse=True)

In [19]:
abundant_sites = {}
for abundance, (gene, location) in sorted_by_abundance:
    assert abundance > 100
    abundant_sites.setdefault(gene, []).append(location)

In [20]:
abundant_sites

{'HLA-B': ['31353871'],
 'RPL30': ['98041720', '98041718'],
 'RPS6': ['19376254'],
 'RPL32': ['12836046', '12836048', '12836025'],
 'TPT1': ['45337176', '45336869'],
 'UBC': ['124911645'],
 'DUSP1': ['172768095'],
 'S100A8': ['153390031'],
 'JUND': ['18279759'],
 'RPL18': ['48615330', '48615327', '48615332'],
 'HLA-C': ['31268748'],
 'TMSB10': ['84906592',
  '84906600',
  '84906593',
  '84906594',
  '84906609',
  '84906610',
  '84906587',
  '84906595',
  '84906605',
  '84906611',
  '84906604',
  '84906590',
  '84906602'],
 'S100A6': ['153534598'],
 'RPL8': ['144789768'],
 'RPS16': ['39433206', '39433223', '39433216'],
 'RPL22': ['6186614'],
 'FOS': ['75282161',
  '75282149',
  '75282144',
  '75282153',
  '75282162',
  '75282163',
  '75282156',
  '75282151',
  '75282148',
  '75282159',
  '75282154',
  '75282160',
  '75282150'],
 'RPS14': ['150444237', '150444228'],
 'RPS4X': ['72272602', '72272611', '72272607'],
 'RPL14': ['40462001',
  '40461996',
  '40461995',
  '40462000',
  '4046199

In [33]:
len(abundant_sites)

137

In [21]:
{i: sites for i, sites in abundant_sites.items() if len(sites) > 1}

{'RPL30': ['98041720', '98041718'],
 'RPL32': ['12836046', '12836048', '12836025'],
 'TPT1': ['45337176', '45336869'],
 'RPL18': ['48615330', '48615327', '48615332'],
 'TMSB10': ['84906592',
  '84906600',
  '84906593',
  '84906594',
  '84906609',
  '84906610',
  '84906587',
  '84906595',
  '84906605',
  '84906611',
  '84906604',
  '84906590',
  '84906602'],
 'RPS16': ['39433206', '39433223', '39433216'],
 'FOS': ['75282161',
  '75282149',
  '75282144',
  '75282153',
  '75282162',
  '75282163',
  '75282156',
  '75282151',
  '75282148',
  '75282159',
  '75282154',
  '75282160',
  '75282150'],
 'RPS14': ['150444237', '150444228'],
 'RPS4X': ['72272602', '72272611', '72272607'],
 'RPL14': ['40462001',
  '40461996',
  '40461995',
  '40462000',
  '40461994',
  '40461998'],
 'S100A4': ['153543620', '153543621', '153543618'],
 'RPS29': ['49583576', '49583578', '49583571', '49583577'],
 'LYZ': ['69354150',
  '69354149',
  '69354169',
  '69354162',
  '69354160',
  '69354171',
  '69354167',
  '69

In [22]:
discover_polya = {}
for gene, gene_data in polya_reads.items():
    count_by_location = discover_polya.setdefault(gene, {})
    for location, sequence_data in gene_data.items():
        UMI_per_location = collections.Counter()
        for CB, UMI, seq in sequence_data:
            UMI_per_location[(CB, UMI)] += 1
        count_by_location[location] = len(UMI_per_location)

In [23]:
# discover_polya["TPT1"]

In [24]:
# polya_reads["TPT1"]["45337190"]

In [25]:
# polya_reads["TPT1"]["45337176"]

In [26]:
# discover_polya["JUN"]

In [27]:
# polya_reads["JUN"]['58780787']

In [28]:
# polya_reads["HLA-DRB1"]['32578768']

In [29]:
# polya_reads["HLA-DRB1"]

In [30]:
# discover_polya["JUN"]

In [31]:
# polya_reads["JUN"]["58781319"]

In [32]:
# polya_reads["JUN"]["58780787"]

# Reference

Match -- 0

Insertion -- 1

Deletion -- 2

Splice -- 3

Softclip -- 4

Hardclip (not good, rerun the alignment) -- 5

```
[(798512, (0,)),
 (143947, (4, 0)),
 (46502, (0, 4)),
 (3855, (4, 0, 4)),
 (3518, (0, 1, 0)),
 (1662, (0, 2, 0)),
 (1380, (0, 3, 0)),
 (133, (4, 0, 1, 0)),
 (123, (0, 1, 0, 4)),
 (109, (4, 0, 3, 0)),
 (101, (0, 3, 0, 4)),
 (62, (0, 2, 0, 4)),
 (46, (4, 0, 2, 0)),
 (11, (4, 0, 3, 0, 4)),
 (9, (0, 3, 0, 3, 0)),
 (8, (0, 3, 0, 1, 0)),
 (6, (4, 0, 1, 0, 4)),
 (4, (0, 1, 0, 1, 0)),
 (3, (4, 0, 2, 0, 4)),
 (3, (0, 1, 0, 2, 0)),
 (2, (0, 2, 0, 1, 0)),
 (1, (4, 0, 1, 0, 2, 0)),
 (1, (0, 2, 0, 2, 0)),
 (1, (0, 1, 0, 3, 0)),
 (1, (0, 1, 0, 1, 0, 4))]
```